### Imports

In [1]:
import nest_asyncio; nest_asyncio.apply()
import re

import json
import os
from dotenv import load_dotenv
load_dotenv()

True

### Function that sends questions to GPT

In [2]:
from openai import OpenAI
client = OpenAI()

model="gpt-4o"
temp = 0
system_message = r"You're an expert in answering quiz questions about phonology. Only answer with the letter and text of the answer which you think is correct. Make sure that the answer is 100% correct because my life depends on it. Answer this question:"

def pingGPT(prompt, model=model):
   # Creating a message as required by the API
   messages = [{"role":"system", "content": system_message},
               {"role": "user", "content": prompt}]
  
   # Calling the ChatCompletion API
   response = client.chat.completions.create(
       model=model,
       messages=messages,
       temperature=temp,
   )

   # Returning the extracted response
   return response.choices[0].message.content

In [3]:
GPT_made_quizzes = ["JSON_quizzes/GPT-made_quizzes/"+f for f in os.listdir("./JSON_quizzes/GPT-made_quizzes") if f.endswith('.json')]
my_own_quizzes = ["JSON_quizzes/my_own_quizzes/"+f for f in os.listdir("./JSON_quizzes/my_own_quizzes") if f.endswith('.json')]
LIST_OF_QUIZZES = GPT_made_quizzes + my_own_quizzes

print(LIST_OF_QUIZZES)
print("number of quizzes:", len(LIST_OF_QUIZZES))

['JSON_quizzes/GPT-made_quizzes/itembank-2.json', 'JSON_quizzes/GPT-made_quizzes/itembank-3.json', 'JSON_quizzes/GPT-made_quizzes/itembank-4.json', 'JSON_quizzes/GPT-made_quizzes/itembank-5.json', 'JSON_quizzes/GPT-made_quizzes/itembank-9.json', 'JSON_quizzes/my_own_quizzes/itembank-2.json', 'JSON_quizzes/my_own_quizzes/itembank-3.json', 'JSON_quizzes/my_own_quizzes/itembank-4.json', 'JSON_quizzes/my_own_quizzes/itembank-5.json', 'JSON_quizzes/my_own_quizzes/itembank-9.json']
number of quizzes: 10


### Open an itembank, send the questions to raw GPT-4o, obtain and save the responses

In [4]:
# save total number of questions and correct answers separately for my own quizzes and GPT-made
total_nquestions_own = 0
total_ncorrects_own = 0
total_nquestions_GPT = 0
total_ncorrects_GPT = 0

itembank_reports = [] # to write the itembank number and number of correct answers.
itembank_reports.append(f"Model: {model}")
itembank_reports.append(f"Temperature: {temp}")
itembank_reports.append(f"System message: {system_message}\n")

for run_n in range(10): # make three passes over all the quizzes
    print(f"\nPass over all the itembanks number {run_n+1}")
    for doc in LIST_OF_QUIZZES:
        with open(doc, 'r', encoding='UTF-8') as file:
            quiz_JSON = json.load(file)
        
        itembank_number = re.search(r"\d[a|b]?(?:\-\d)?", doc).group()
        quiz_type = "own_questions" if re.search(r"my_own_quizzes", doc) else "GPT-made_questions" # returns None for GPT-made questions
        print(f"----------processing itembank-{itembank_number}-{quiz_type}----------")
        
        this_ncorrects = 0 # for correct answers in each separate itembank
        this_nquestions = 0

        # main loop that sends requests and saves answers
        for question in quiz_JSON:
            full_prompt = question['question']+'\n' + '\n'.join(question['answers'])
            given_answer = pingGPT(full_prompt)  # Get the response from GPT
            question["GPT's response"] = given_answer  # Append the response to the question
            question["correct"] = given_answer==question["correct answer(s)"] # stores whether the answer is correct
            
            # count how many total responses are correct for own questions and GPT-made
            if quiz_type == "own_questions":
                if given_answer==question["correct answer(s)"]: 
                    total_ncorrects_own+=1
                    this_ncorrects+=1
                total_nquestions_own+=1
                this_nquestions+=1
            else:
                if given_answer==question["correct answer(s)"]: 
                    total_ncorrects_GPT+=1
                    this_ncorrects+=1
                total_nquestions_GPT+=1
                this_nquestions+=1
        
        itembank_reports.append(f"itembank-{itembank_number}_{quiz_type}:\t\tCorrect answers {this_ncorrects} / total questions {this_nquestions} * 100 = {round(this_ncorrects/this_nquestions*100, 2)}%")

        # Save the updated JSON data to a new file
        with open(f'./results/raw_gpt_responses/{quiz_type}_responses-{itembank_number}.json', 'w') as outfile:
            json.dump(quiz_JSON, outfile, indent=2)
    itembank_reports.append(" ")
    
    print(f"Overall correct answers for GPT-made {total_ncorrects_GPT} / total questions {total_nquestions_GPT} * 100 = {round(total_ncorrects_GPT/total_nquestions_GPT*100, 2)}%\n")
    print(f"Overall correct answers for own {total_ncorrects_own} / total questions {total_nquestions_own} * 100 = {round(total_ncorrects_own/total_nquestions_own*100, 2)}%\n")

itembank_reports.append(f"\nOverall correct answers for GPT-made {total_ncorrects_GPT} / total questions {total_nquestions_GPT} * 100 = {round(total_ncorrects_GPT/total_nquestions_GPT*100, 2)}%")
itembank_reports.append(f"Overall correct answers for own {total_ncorrects_own} / total questions {total_nquestions_own} * 100 = {round(total_ncorrects_own/total_nquestions_own*100, 2)}%\n")
itembank_reports.append("\n------------------------------------------------------------------------\n\n")

# save the prompt and number of correct questions
with open(f"./results/raw_{model}_results.txt", 'a', encoding='UTF-8') as report_file:
    report_file.write('\n'.join(itembank_reports))


Pass over all the itembanks number 1
----------processing itembank-2-GPT-made_questions----------
----------processing itembank-3-GPT-made_questions----------
----------processing itembank-4-GPT-made_questions----------
----------processing itembank-5-GPT-made_questions----------
----------processing itembank-9-GPT-made_questions----------
----------processing itembank-2-own_questions----------
----------processing itembank-3-own_questions----------
----------processing itembank-4-own_questions----------
----------processing itembank-5-own_questions----------
----------processing itembank-9-own_questions----------
Overall correct answers for GPT-made 169 / total questions 219 * 100 = 77.17%

Overall correct answers for own 219 / total questions 300 * 100 = 73.0%


Pass over all the itembanks number 2
----------processing itembank-2-GPT-made_questions----------
----------processing itembank-3-GPT-made_questions----------
----------processing itembank-4-GPT-made_questions----------
----